In [1]:
import pandas as pd

In [2]:
# read the scrobbles data set
df = pd.read_csv('data/lastfm_scrobbles.csv', parse_dates=True, infer_datetime_format=True)
df = df.drop('timestamp', axis=1)
df.head()

,artist,album,track,datetime
0,Matt Dunkley,Six Cycles,Cycle 1,2016-03-25 00:14:58
1,Cremation Lily,Radiance and Instability,Radiance and Instability,2016-03-25 00:10:34
2,Jameszoo,NaN,Flake,2016-03-25 00:06:41
3,Vanessa Amara,NaN,Untitled 2,2016-03-25 00:03:07
4,Dalhous,The Composite Moods Collection Vol.1: House Nu...,Content For Feelings,2016-03-24 23:59:39


In [3]:
df.index.is_unique

True

In [4]:
# according to last.fm web site, i have ~259,000 scrobbles - missing the ones from before 2007
len(df)

224258

In [5]:
# how many artists?
len(df['artist'].unique())

15016

In [6]:
# most listened to artists
df['artist'].value_counts().head(10)

David Bowie            2511
The Kinks              2208
The Beatles            1912
Bob Dylan              1665
Radiohead              1663
Yo La Tengo            1533
The Rolling Stones     1488
Belle and Sebastian    1430
Pavement               1418
Frank Zappa            1334
Name: artist, dtype: int64

In [7]:
# most listened to albums
df.apply(lambda row: '{} - {}'.format(row['artist'], row['album']), axis=1).value_counts().head(15)

The Kinks - The Kink Kronikles                        746
Fleet Foxes - Fleet Foxes                             544
The Residents - The Commercial Album                  530
Frank Sinatra - Nothing But The Best                  516
Madvillain - Madvillainy                              493
Devo - Pioneers Who Got Scalped                       492
Robert Johnson - King Of The Delta Blues              488
Surfer Blood - Astro Coast                            484
The Clash - London Calling                            484
Vampire Weekend - Vampire Weekend                     467
Os Mutantes - Os Mutantes                             466
Frank Black - Teenager of the Year                    464
Grizzly Bear - Veckatimest                            451
Neutral Milk Hotel - In the Aeroplane Over the Sea    441
Girl Talk - Feed the Animals                          441
dtype: int64

In [8]:
# most listened to tracks
df.apply(lambda row: '{} - {}'.format(row['artist'], row['track']), axis=1).value_counts().head(15)

Animal Collective - Summertime Clothes     91
Surfer Blood - Floating Vibes              81
Can - Vitamin C                            75
The Dismemberment Plan - Gyroscope         75
Fleet Foxes - White Winter Hymnal          73
Can - I'm So Green                         72
Surfer Blood - Take It Easy                72
Wu-Tang Clan - Shame on a Nigga            71
The Kinks - Days                           70
Panda Bear - Ponytail                      69
The Unicorns - Ready To Die                69
Thelonious Monk - Epistrophy               68
The Jam - Pretty Green                     63
Can - Sing Swan Song                       62
The Velvet Underground - Pale Blue Eyes    62
dtype: int64